In [36]:
# # based on the post here: https://www.kaggle.com/c/petfinder-pawpularity-score/discussion/275094

import sys
sys.path.append("../")
# sys.path.append("../input/tez-lib/")
# sys.path.append("../input/timmmaster/")
# sys.path.append("../input/matsuda-utils")

In [37]:
import numpy as np, pandas as pd
from glob import glob
import shutil, os
import pickle
import matplotlib.pyplot as plt
from sklearn.model_selection import GroupKFold
from sklearn.decomposition import IncrementalPCA
from tqdm.notebook import tqdm
import seaborn as sns
import PIL.Image as Image
import torch
from torch.utils.data import Dataset, DataLoader
import cv2
import time
import pandas_profiling as pdp
from pathlib import Path
from sklearn.model_selection import StratifiedKFold
from utils.util import *
from utils.losses import *
import torch.nn as nn
import transformers as T
import albumentations
import pandas as pd
import cv2
import numpy as np
import timm
import torch.nn as nn
from sklearn import metrics
import torch.nn.functional as F
import torch
from tez.callbacks import EarlyStopping
from tqdm import tqdm
from collections import OrderedDict
import math
import tez
import torch.optim as optim

In [38]:
class CONFIG:
    DATA_PATH = Path('../input/petfinder-pawpularity-score')
    MODEL_NAME = 'swin_large_patch4_window12_384'
    MODEL_PATH = Path(f'../input/pretrained_models/swin_large_patch4_window12_384_add_petcategory/')
    batch_size = 32
    image_size = 384
    fold = 5
    device='cuda'

In [ ]:
from u2net_test import extract
# extract(str(CONFIG.DATA_PATH) + '/train',str(CONFIG.DATA_PATH)+ '/train_U2NET')
extract(str(CONFIG.DATA_PATH) +'/test', str(CONFIG.DATA_PATH) +  '/test_U2NET')
# extract(str(CONFIG.NO_LABEL_DATA_PATH) + '/train_images',str(CONFIG.NO_LABEL_DATA_PATH)+ '/train_U2NET')
# extract(str(CONFIG.NO_LABEL_DATA_PATH) +'/test_images', str(CONFIG.NO_LABEL_DATA_PATH) +  '/test_U2NET')

In [39]:
train_df = pd.read_csv(CONFIG.DATA_PATH / 'train.csv')
train_df['path'] = train_df['Id'].map(lambda x:str(CONFIG.DATA_PATH/'train'/x)+'.jpg')
train_df['mask_path'] = train_df['Id'].map(lambda x:str(CONFIG.DATA_PATH/'train_U2NET'/x)+'.jpg')
train_df['image_size'] = train_df['path'].apply(lambda image_id : Image.open(image_id).size)
train_df['width'] = train_df['image_size'].apply(lambda x: x[0])
train_df['height'] = train_df['image_size'].apply(lambda x: x[1])

In [40]:
test_aug = albumentations.Compose(
    [
        albumentations.Resize(CONFIG.image_size, CONFIG.image_size, p=1),
        albumentations.Normalize(
            mean=[0.485, 0.456, 0.406],
            std=[0.229, 0.224, 0.225],
            max_pixel_value=255.0,
            p=1.0,
        ),
    ],
    p=1.0,
)

In [41]:
class PawpularDataset:
    def __init__(self, df, targets, augmentations):
        self.image_paths = df['path'].tolist()
        self.mask_paths = df['mask_path'].tolist()
        self.targets = targets
        if self.targets is None:
            self.targets = torch.ones(len(self.image_paths))
        self.augmentations = augmentations
        
    def __len__(self):
        return len(self.image_paths)
    
    def __getitem__(self, item):
        image = cv2.imread(self.image_paths[item])
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        mask = cv2.imread(self.mask_paths[item])
        mask = cv2.cvtColor(mask, cv2.COLOR_BGR2GRAY)
        h_max = 0
        h_min = mask.shape[0]
        w_max = 0
        w_min = mask.shape[1]
        idx = np.where(mask > 127.5)
        try:
            h_max = idx[0].min()
            h_min = idx[0].max()
            w_max = idx[1].min()
            w_min = idx[1].max()
        except:
            pass
        image = image[h_max:h_min, w_max:w_min,:]
        mask = mask[h_max:h_min, w_max:w_min]
        
        if self.augmentations is not None:
            augmented = self.augmentations(image=image, mask=mask)
            image = augmented["image"]
            mask = augmented["mask"]
        
        targets = self.targets[item]
        image = np.transpose(image, (2, 0, 1)).astype(np.float32)
        return {
            "image": torch.tensor(image, dtype=torch.float),
            "mask" : torch.tensor(mask, dtype=torch.float),
            "targets": torch.tensor(targets, dtype=torch.float)
        }

In [42]:
class PawpularModel(nn.Module):
    def __init__(self, pet_classify_model, model_name):
        super().__init__()
        self.pet_classify_model = pet_classify_model
        self.pet_classify_model.requires_grad = False
        self.model = timm.create_model(model_name, pretrained=CONFIG.pretrained, in_chans=3)
        self.model.head = nn.Linear(self.model.head.in_features, 128)
        self.dropout = nn.Dropout(0.1)
        self.dense1 = nn.Linear(128+37, 64)
        self.dense2 = nn.Linear(64, 1)

    def forward(self, image):
        p = self.pet_classify_model(F.adaptive_avg_pool2d(image, (224,224)))
        p = torch.softmax(p, dim=1)
        x = self.model(image)
        x = self.dropout(x)
        x = torch.cat([x, p], dim=1)
        x = self.dense1(x)
        x = self.dense2(x)
        return torch.sigmoid(x.squeeze(1))
    
class pet_categor_extract_model(nn.Module):
    def __init__(self,class_num):
        super().__init__()
        self.model = timm.create_model('efficientnet_b0', pretrained=True, in_chans=3)
        self.model.classifier = nn.Linear(self.model.classifier.in_features, 128)
        self.dropout = nn.Dropout(0.1)
        self.dense = nn.Linear(128
                               , class_num)

    def forward(self, image):
        x = self.model(image)
        x = self.dropout(x)
        x = self.dense(x)
        return x.squeeze(1)

In [43]:
def inference():
    predictions = []
    dense_features = [
        'Subject Focus', 'Eyes', 'Face', 'Near', 'Action', 'Accessory',
        'Group', 'Collage', 'Human', 'Occlusion', 'Info', 'Blur'
    ]
    test_dataset = PawpularDataset(
        test_df, targets=None,
        augmentations=test_aug
    )
    test_loader = DataLoader(
        test_dataset, batch_size=16, shuffle=False, num_workers=4, pin_memory=True
    )

    for fold in range(5):
        LOGGER.info(f"========== model: {CONFIG.MODEL_NAME} fold: {fold} inference ==========")
        cl_model = pet_categor_extract_model(class_num=37)
        cl_model.to(CONFIG.device)
        cl_model.load_state_dict(fix_model_state_dict(torch.load('../input/pretrained_models/efficientnet_b0_Oxford_classifier_size_224.pth')["model"]))

        model = PawpularModel(cl_model, model_name=CONFIG.MODEL_NAME)
        model.to(CONFIG.device)
        if torch.cuda.device_count()>1:
            model=nn.DataParallel(model)
            model.load_state_dict(torch.load(CONFIG.OUTPUT_DIR / f"{CONFIG.MODEL_NAME}_{fold}_best.pth")["model"])
        else:
            model.load_state_dict(fix_model_state_dict(torch.load(CONFIG.OUTPUT_DIR / f"{CONFIG.MODEL_NAME}_{fold}_best.pth")["model"]))
        model.eval()
        preds = []
        for i, data in tqdm(enumerate(test_loader), total=len(test_loader)):
            img, mask, target = data['image'], data['mask'], data['targets']
            img = img.to(CONFIG.device)
            target = target.to(CONFIG.device)
            
            img = (img * (mask>0.5).unsqueeze(1)).to(device)
            with torch.no_grad():
                y_preds = model(img)
            preds.append(y_preds.to("cpu").numpy())
        preds = np.concatenate(preds)
        predictions.append(preds)
    predictions = np.mean(predictions, axis=0)
    return predictions * 100

In [44]:
def main():
    # Inference
    predictions = inference()
    # submission
    submission = test_df.copy()
    submission["Pawpularity"] = predictions
    submission = submission[["Id", "Pawpularity"]]
    submission.to_csv("submission.csv", index=False)

In [45]:
if __name__ == "__main__":
    main()

100%|██████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  3.55it/s]
Exception in thread QueueFeederThread:
Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/multiprocessing/queues.py", line 232, in _feed
    close()
  File "/opt/conda/lib/python3.7/multiprocessing/connection.py", line 177, in close
    self._close()
  File "/opt/conda/lib/python3.7/multiprocessing/connection.py", line 361, in _close
    _close(self._handle)
OSError: [Errno 9] Bad file descriptor

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/opt/conda/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/conda/lib/python3.7/multiprocessing/queues.py", line 263, in _feed
    queue_sem.release()
ValueError: semaphore or lock released t